In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../firedetect')

In [3]:
from dataset import load_dataset
from model import Model

# Check the no. of train samples

In [4]:
!ls '../data/fire-dataset-dunnings/images-224x224/train/fire' | wc

  12550   12550  285998


# Set hyperparams

In [5]:
WEIGHTS_PATH = '../weights'
BACKBONES = ['resnet18','resnet34','resnet50','resnet101', 'densenet121', 'mobilenet']
BACKBONES = ['resnet18','resnet50']
BATCH_SIZE = 16
EPOCHS = 10

# Build data loaders

In [6]:
dataset_paths = {'mine': '../data/fire_aerial2k_dataset/',
                 'dunnings': '../data/fire-dataset-dunnings/images-224x224/train/',
                 'dunnings_test': '../data/fire-dataset-dunnings/images-224x224/test/'}

# transform
tr = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
                            torchvision.transforms.ToTensor()])

# train, valid
train_loader, valid_loader = load_dataset(dataset_paths['dunnings'], batch_size=BATCH_SIZE)


# test
test_dataset = torchvision.datasets.ImageFolder(root=dataset_paths['dunnings_test'],
                                                transform=tr)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=True
)

# completely memory extravagant, but can be faster if we have tons of RAM
# train, valid = list(train), list(valid) 

print(f'Loaded {len(train_loader)} training batches with {len(train_loader) * BATCH_SIZE} samples')
print(f'Loaded {len(valid_loader)} validation batches with {len(valid_loader) * BATCH_SIZE} samples')
print(f'Loaded {len(test_loader)} test batches with {len(test_loader) * BATCH_SIZE} samples')

Loaded 1317 training batches with 21072 samples
Loaded 147 validation batches with 2352 samples
Loaded 184 test batches with 2944 samples


In [7]:
def accuracy_gpu(pred, truth):
    agreeing = pred.eq(truth)
    acc = agreeing.sum().double()/agreeing.numel()
    return float(acc)

In [ ]:
import torch
device = torch.device("cuda:0")
is_validating = True
is_testing = False

history = {
    'train_samples': [],
    'train_acc': [],
    'valid_acc': [],
    'test_acc': [],
    'loss': []
}


for b in BACKBONES:

    import torch.optim as optim

    m = Model(backbone=b)
    m = m.to(device)

    criterion = torch.nn.BCELoss()

    for epoch in range(1,EPOCHS+1): # epochs
        m.train()
        optimizer = optim.Adam(m.parameters(), lr= 1e-4 if epoch <5 else 1e-5, weight_decay=1e-3)
        
        running_loss = []
        running_acc = []

        # epoch training
        for i, data in enumerate(train_loader):

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # get the inputs; data is a list of [inputs, labels]
            X = data[0].to(device)
            y = data[1].to(device)


            # forward pass
            scores = m(X).squeeze()
            pred = scores > 0.5
            
            # backprop
            loss = criterion(scores, y.type_as(scores))
            loss.backward()
            optimizer.step()
            running_loss.append(loss.item())
            
            # metrics
            acc = accuracy_gpu(pred, y)
            running_acc.append(acc)
            
            # print metrics every batches
            if i%20 == 0:
                print(f'epoch: {epoch:02d}, batch: {i:03d}, \
                loss: {np.mean(running_loss):.3f}, training accuracy: {np.mean(running_acc):.3f}')

                history['loss'].append(np.mean(running_loss))
                history['train_samples'].append(epoch*len(train_loader)+i)
                history['train_acc'].append(np.mean(running_acc))
                
                
        # on epoch end:
        if is_validating:
            # epoch validation
            with torch.no_grad():
                m.eval()
                running_acc = []
                # epoch validation
                for i, data in enumerate(valid_loader):
                    # get the inputs; data is a list of [inputs, labels]
                    X = data[0].to(device)
                    y = data[1].to(device)

                    scores = m(X).squeeze()
                    pred = scores > 0.5
                    acc = accuracy_gpu(pred, y)
                    running_acc.append(acc)
                    
                va = round(np.mean(running_acc), 4)
                print(f'validation accuracy {va}')
                history['valid_acc'].append(va)
        else:
            va='-1'

        if is_testing:
            with torch.no_grad():
                m.eval()
                running_acc = []
                # epoch testing
                for i, data in enumerate(test_loader):
                    # get the inputs; data is a list of [inputs, labels]
                    X = data[0].to(device)
                    y = data[1].to(device)
                    
                    scores = m(X).squeeze()
                    pred = scores > 0.5
                    acc = accuracy_gpu(pred, y)
                    running_acc.append(acc)
                    
                tst = round(np.mean(running_acc), 4)
                print(f'test_accuracy {tst}')
                history['test_acc'].append(tst)
        else:
            tst = '-1'

        fname =  f'{WEIGHTS_PATH}/{b}-epoch-{epoch}-valid_acc={va}-test_acc={tst}.pt'
        print(f'Saved {fname}')
        torch.save(m, fname)


    print(f'Finished Training: {b}')

epoch: 01, batch: 000,                 loss: 0.749, training accuracy: 0.500
epoch: 01, batch: 020,                 loss: 0.340, training accuracy: 0.836
epoch: 01, batch: 040,                 loss: 0.267, training accuracy: 0.887
epoch: 01, batch: 060,                 loss: 0.225, training accuracy: 0.910
epoch: 01, batch: 080,                 loss: 0.199, training accuracy: 0.922
epoch: 01, batch: 100,                 loss: 0.185, training accuracy: 0.928
epoch: 01, batch: 120,                 loss: 0.172, training accuracy: 0.932
epoch: 01, batch: 140,                 loss: 0.167, training accuracy: 0.934
epoch: 01, batch: 160,                 loss: 0.157, training accuracy: 0.937
epoch: 01, batch: 180,                 loss: 0.156, training accuracy: 0.939
epoch: 01, batch: 200,                 loss: 0.154, training accuracy: 0.938
epoch: 01, batch: 220,                 loss: 0.151, training accuracy: 0.941
epoch: 01, batch: 240,                 loss: 0.147, training accuracy: 0.943

epoch: 02, batch: 780,                 loss: 0.042, training accuracy: 0.987
epoch: 02, batch: 800,                 loss: 0.043, training accuracy: 0.987
epoch: 02, batch: 820,                 loss: 0.043, training accuracy: 0.987
epoch: 02, batch: 840,                 loss: 0.043, training accuracy: 0.986
epoch: 02, batch: 860,                 loss: 0.043, training accuracy: 0.986
epoch: 02, batch: 880,                 loss: 0.044, training accuracy: 0.986
epoch: 02, batch: 900,                 loss: 0.043, training accuracy: 0.986
epoch: 02, batch: 920,                 loss: 0.043, training accuracy: 0.986
epoch: 02, batch: 940,                 loss: 0.043, training accuracy: 0.987
epoch: 02, batch: 960,                 loss: 0.043, training accuracy: 0.986
epoch: 02, batch: 980,                 loss: 0.044, training accuracy: 0.986
epoch: 02, batch: 1000,                 loss: 0.045, training accuracy: 0.986
epoch: 02, batch: 1020,                 loss: 0.044, training accuracy: 0.9

epoch: 04, batch: 220,                 loss: 0.039, training accuracy: 0.988
epoch: 04, batch: 240,                 loss: 0.037, training accuracy: 0.988
epoch: 04, batch: 260,                 loss: 0.038, training accuracy: 0.988
epoch: 04, batch: 280,                 loss: 0.037, training accuracy: 0.988
epoch: 04, batch: 300,                 loss: 0.037, training accuracy: 0.988
epoch: 04, batch: 320,                 loss: 0.036, training accuracy: 0.988
epoch: 04, batch: 340,                 loss: 0.035, training accuracy: 0.988
epoch: 04, batch: 360,                 loss: 0.034, training accuracy: 0.989
epoch: 04, batch: 380,                 loss: 0.034, training accuracy: 0.989
epoch: 04, batch: 400,                 loss: 0.035, training accuracy: 0.989
epoch: 04, batch: 420,                 loss: 0.033, training accuracy: 0.989
epoch: 04, batch: 440,                 loss: 0.034, training accuracy: 0.989
epoch: 04, batch: 460,                 loss: 0.033, training accuracy: 0.989

epoch: 05, batch: 1000,                 loss: 0.027, training accuracy: 0.991
epoch: 05, batch: 1020,                 loss: 0.027, training accuracy: 0.991
epoch: 05, batch: 1040,                 loss: 0.028, training accuracy: 0.991
epoch: 05, batch: 1060,                 loss: 0.028, training accuracy: 0.991
epoch: 05, batch: 1080,                 loss: 0.028, training accuracy: 0.991
epoch: 05, batch: 1100,                 loss: 0.028, training accuracy: 0.991
epoch: 05, batch: 1120,                 loss: 0.028, training accuracy: 0.990
epoch: 05, batch: 1140,                 loss: 0.028, training accuracy: 0.990
epoch: 05, batch: 1160,                 loss: 0.029, training accuracy: 0.990
epoch: 05, batch: 1180,                 loss: 0.029, training accuracy: 0.990
epoch: 05, batch: 1200,                 loss: 0.030, training accuracy: 0.990
epoch: 05, batch: 1220,                 loss: 0.030, training accuracy: 0.990
epoch: 05, batch: 1240,                 loss: 0.030, training ac

epoch: 07, batch: 440,                 loss: 0.005, training accuracy: 0.998
epoch: 07, batch: 460,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 480,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 500,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 520,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 540,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 560,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 580,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 600,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 620,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 640,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 660,                 loss: 0.005, training accuracy: 0.999
epoch: 07, batch: 680,                 loss: 0.005, training accuracy: 0.999

epoch: 08, batch: 1220,                 loss: 0.003, training accuracy: 0.999
epoch: 08, batch: 1240,                 loss: 0.003, training accuracy: 0.999
epoch: 08, batch: 1260,                 loss: 0.003, training accuracy: 0.999
epoch: 08, batch: 1280,                 loss: 0.003, training accuracy: 0.999
epoch: 08, batch: 1300,                 loss: 0.003, training accuracy: 0.999
validation accuracy 0.994
Saved ../weights/resnet18-epoch-7-valid_acc=0.994-test_acc=-1.pt
epoch: 09, batch: 000,                 loss: 0.000, training accuracy: 1.000
epoch: 09, batch: 020,                 loss: 0.001, training accuracy: 1.000
epoch: 09, batch: 040,                 loss: 0.002, training accuracy: 1.000
epoch: 09, batch: 060,                 loss: 0.005, training accuracy: 0.999
epoch: 09, batch: 080,                 loss: 0.004, training accuracy: 0.999
epoch: 09, batch: 100,                 loss: 0.003, training accuracy: 0.999
epoch: 09, batch: 120,                 loss: 0.003, train

In [ ]:
# for history in histories:
plt.figure()
plt.plot(history['train_samples'], history['loss'])

In [ ]:
plt.plot(history['train_samples'], history['train_acc'])

In [ ]:
history['valid_acc']

In [ ]:
plt.scatter(np.array(history['train_samples'])/len(train_loader), history['train_acc'])
plt.scatter(list(range(10)), history['valid_acc'])
plt.scatter(list(range(10)), history['test_acc'])

In [ ]:
for k,v in histories.items():
    print(v['train_acc'])

In [ ]:
history.keys()

